In [14]:
# grader-required-cell

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [15]:
# grader-required-cell

# Load the data

# Get current working directory
current_dir = os.getcwd()

# Append data/mnist.npz to the previous path to get the full path
data_path = os.path.join(current_dir, "mnist.npz")

# Get only training set
(training_images, training_labels), _ = tf.keras.datasets.mnist.load_data(path=data_path)


In [16]:
# grader-required-cell

# GRADED FUNCTION: reshape_and_normalize

def reshape_and_normalize(images):

    # Reshape the images to add an extra dimension
    images = np.expand_dims(images, axis=-1)
    
    # Normalize pixel values
    images = images / 255.0

    return images

In [17]:
# grader-required-cell

# Reload the images in case you run this cell multiple times
(training_images, _), _ = tf.keras.datasets.mnist.load_data(path=data_path)

# Apply your function
training_images = reshape_and_normalize(training_images)

print(f"Maximum pixel value after normalization: {np.max(training_images)}\n")
print(f"Shape of training set after reshaping: {training_images.shape}\n")
print(f"Shape of one image after reshaping: {training_images[0].shape}")


Maximum pixel value after normalization: 1.0

Shape of training set after reshaping: (60000, 28, 28, 1)

Shape of one image after reshaping: (28, 28, 1)


In [18]:
# grader-required-cell

# GRADED CLASS: myCallback

# Remember to inherit from the correct class
class myCallback(tf.keras.callbacks.Callback):
    # Define the method that checks the accuracy at the end of each epoch
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.995:
            self.model.stop_training = True


In [19]:
# grader-required-cell

# GRADED FUNCTION: convolutional_model
def convolutional_model():

    # Define the model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),

  # Add the same layers as before
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
        
    return model

In [ ]:
# grader-required-cell

# Save your untrained model
model = convolutional_model()

# Get number of weights
model_params = model.count_params()

# Unit test to limit the size of the model
assert model_params < 1000000, (
    f'Your model has {model_params:,} params. For successful grading, please keep it ' 
    f'under 1,000,000 by reducing the number of units in your Conv2D and/or Dense layers.'
)

# Instantiate the callback class
callbacks = myCallback()

# Train your model (this can take up to 5 minutes)
history = model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 38s 19ms/step - loss: 0.1482 - accuracy: 0.9549
Epoch 2/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0448 - accuracy: 0.9857
Epoch 3/10
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0319 - accuracy: 0.9898
Epoch 4/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0226 - accuracy: 0.9929
Epoch 5/10
1142/1875 [=================>............] - ETA: 14s - loss: 0.0153 - accuracy: 0.9947

In [ ]:
# grader-required-cell

print(f"Your model was trained for {len(history.epoch)} epochs")

In [ ]:
if not "accuracy" in history.model.metrics_names:
    print("Use 'accuracy' as metric when compiling your model.")
else:
    print("The metric was correctly defined.")